# Download Any Dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qa1LKc9g1e9QJ7VWWpFU")
project = rf.workspace("webdesign").project("website-3uipa")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qa1LKc9g1e9QJ7VWWpFU")
project = rf.workspace("visualgrounding").project("object-detection-9ba1m")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qa1LKc9g1e9QJ7VWWpFU")
project = rf.workspace("object-detetction-0gxei").project("object-detection-child-subset")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qa1LKc9g1e9QJ7VWWpFU")
project = rf.workspace("visualgrounding").project("object-detection-parent-data")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qa1LKc9g1e9QJ7VWWpFU")
project = rf.workspace("shubham-choudhary-lfxur").project("all_elements")
version = project.version(2)
dataset = version.download("yolov8")


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qa1LKc9g1e9QJ7VWWpFU")
project = rf.workspace("roboflow-gw7yv").project("website-screenshots")
version = project.version(1)
dataset = version.download("yolov8")


# Train Model Thourgh CMD

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/Users/jatavathpavannaik/Documents/python/Computer_Vision/YOLOV8/dataset-v2-1/data.yaml epochs=100 imgsz=224 plots=True

In [ ]:
import torch
from ultralytics import YOLO


model = YOLO("/Users/jatavathpavannaik/Documents/python/Computer_Vision/YOLOV8/runs/detect/train/weights/last.pt")

model.train(resume=True)

# Predict Through CMD

In [ ]:
!yolo task=detect mode=predict model=/Users/jatavathpavannaik/Documents/python/Computer_Vision/YOLOV8/runs/detect/train_cpu/weights/best.pt conf=0.1 source=/Users/jatavathpavannaik/Documents/python/Computer_Vision/Booking.com.png

# Predict Using Python

In [ ]:
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO("/Users/jatavathpavannaik/Documents/python/Computer_Vision/Yolo/runs/detect/train/weights/best.pt")

# Make predictions
results = model.predict("/Users/jatavathpavannaik/Documents/python/Computer_Vision/airbnb_1.png")

boxes = results[0].boxes.xyxy.tolist()
classes = results[0].boxes.cls.tolist()
names = results[0].names
confidences = results[0].boxes.conf.tolist()

# Iterate through the results
for box, cls, conf in zip(boxes, classes, confidences):
    x1, y1, x2, y2 = box
    confidence = conf
    detected_class = cls
    name = names[int(cls)]
    


# Use Case

In [ ]:
import cv2
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO("/Users/jatavathpavannaik/Documents/python/Computer_Vision/YOLOV8/runs/detect/train_cpu/weights/best.pt")

# Open video capture
cap = cv2.VideoCapture('/Users/jatavathpavannaik/Documents/python/Computer_Vision/Bookin.com_Staganent.mov')  # Replace 'path_to_video.mp4' with your video file
  # Replace 'path_to_video.mp4' with your video file
fps = cap.get(cv2.CAP_PROP_FPS)
# Create a list to store areas of bounding boxes for each frame
all_areas = []

frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_number += 1
    
    # Make predictions
    results = model.predict(frame)
    
    # Extract bounding box information
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()
    
    # Draw bounding boxes on the frame and calculate areas
    areas=[]
    for box, cls, conf in zip(boxes, classes, confidences):
        x1, y1, x2, y2 = box
        class_name = names[int(cls)]
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, f'{class_name} {conf:.2f}', (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Calculate area
        area = (x2 - x1) * (y2 - y1)
        areas.append(area)
    
    # Store areas of bounding boxes for this frame
    all_areas.append((frame_number, areas))
    
    # Display the frame with bounding boxes
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
all_areas

In [ ]:
import pandas as pd

# Assuming all_areas is a list of tuples where each tuple contains (frame_name, list_of_areas)
sum_data = [(tup[0], sum(tup[1]), len(tup[1]), tup[1]) for tup in all_areas]

# Create DataFrame
df = pd.DataFrame(sum_data, columns=['Frame', 'Total_Area', 'No_of_Bounding_Boxes', 'Individual_Areas'])


In [ ]:
df.to_clipboard()

In [ ]:
def delete_frames_in_duplicate_series(df, duplicate_series, frame_column='Frame'):
    df_cleaned = df.copy()
    for start, end in duplicate_series:
        frames_to_delete = list(range(start, end + 1))
        df_cleaned = df_cleaned[~df_cleaned[frame_column].isin(frames_to_delete)]
    return df_cleaned

# Example usage:
cleaned_df = delete_frames_in_duplicate_series(df, duplicate_series)



In [ ]:
cleaned_df.to_clipboard()

In [ ]:
cleaned_df

In [ ]:
def find_duplicate_frames(df):
        start_index = None
        end_index = None
        duplicate_series = []

        for index, row in df.iterrows():
            if index > 0:
                prev_row = df.iloc[index - 1]
                if row['No_of_Bounding_Boxes'] == prev_row['No_of_Bounding_Boxes']:
                    areas_current = row['Individual_Areas']
                    areas_prev = prev_row['Individual_Areas']
                    all_diffs_less_than_100 = all(abs(area_curr - area_prev) < 100 for area_curr, area_prev in zip(areas_current, areas_prev))
                    if all_diffs_less_than_100:
                        if start_index is None:
                            start_index = index - 1
                        end_index = index
                else:
                    if start_index is not None:
                        duplicate_series.append((start_index, end_index))
                        start_index = None
                        end_index = None

        if start_index is not None:
            duplicate_series.append((start_index, end_index))

        if duplicate_series:
            return duplicate_series
        else:
            print("No Duplicated Frames Found")
duplicate_series = find_duplicate_frames(df)

In [ ]:
duplicate_series

In [ ]:
import random

frames = []
threshold_seconds = 5  # Example threshold in seconds
threshold_frames = int(threshold_seconds * fps)  # Convert threshold to frames

for start, end in duplicate_series:
    if end - start > threshold_frames:
        random_index = random.randint(start, end)
        random_row = df.iloc[random_index]
        frames.append(random_row)

if frames:
    req_frames = pd.DataFrame(frames)


In [ ]:
duplicate_series

In [ ]:
req_frames

In [ ]:
import random

def select_frames_with_threshold(duplicate_series, df, fps, threshold_seconds):
    frames = []
    threshold_frames = int(threshold_seconds * fps)  # Convert threshold to frames

    for start, end in duplicate_series:
        if end - start > threshold_frames:
            random_index = random.randint(start, end)
            random_row = df.iloc[random_index]
            frames.append(random_row)

    if frames:
        req_frames = pd.DataFrame(frames)
        return req_frames
    else:
        return None

In [ ]:
import pandas as pd

# Assuming df is your DataFrame containing the data
desired_rows = []

for index, row in df.iterrows():
    if index > 0:  # Skip the first row
        prev_row = df.iloc[index - 1]  # Get the previous row
        if row['No_of_Bounding_Boxes'] == prev_row['No_of_Bounding_Boxes']:
            areas_current = row['Individual_Areas']
            areas_prev = prev_row['Individual_Areas']
            all_diffs_less_than_100 = all(abs(area_curr - area_prev) < 100 for area_curr, area_prev in zip(areas_current, areas_prev))
            if all_diffs_less_than_100:
                desired_rows.append(row)

if desired_rows:
    desired_df = pd.DataFrame(desired_rows)
    print(desired_df)
else:
    print("No rows found matching the criteria.")


In [ ]:
numbers=desired_df["Frame"].tolist()

In [ ]:
def find_consecutive_sequence_indices(numbers):
    start_index = 0
    end_index = 0
    result = []

    for i in range(1, len(numbers)):
        if numbers[i] == numbers[i - 1] + 1:
            end_index = i
        else:
            if end_index - start_index > 0:
                result.append((start_index, end_index))
            start_index = i
            end_index = i

    # Check if the last sequence is consecutive
    if end_index - start_index > 0:
        result.append((start_index, end_index))

    return result

# Example usage:
sequences = find_consecutive_sequence_indices(numbers)


In [ ]:
Frames_to_Delete=desired_df["Frame"].astype(int).to_list()

In [ ]:
import cv2

def extract_frames(video_path, frame_numbers):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_number in frame_numbers:
            frames.append(frame)
        
        if len(frames) == len(frame_numbers):
            break
    
    cap.release()
    return frames

# Example usage
video_path = "/Users/jatavathpavannaik/Documents/python/Computer_Vision/Booking.com.mov"
frame_numbers = Frames_to_Delete
extracted_frames = extract_frames(video_path, frame_numbers)

# Do something with the extracted frames, such as saving them or further processing
# For example, to save the frames as images:
for i, frame in enumerate(extracted_frames):
    cv2.imwrite(f"/Users/jatavathpavannaik/Documents/python/Computer_Vision/Exports/Extracted_Frames/frame_{frame_numbers[i]}.jpg", frame)


In [ ]:
import cv2

def delete_frames(input_video_path, output_video_path, frames_to_delete):
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    frame_number = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_number not in frames_to_delete:
            out.write(frame)

        frame_number += 1

    cap.release()
    out.release()

# Example usage
input_video_path = "/Users/jatavathpavannaik/Documents/python/Computer_Vision/Bookin.com_Staganent.mov"
output_video_path = "/Users/jatavathpavannaik/Documents/python/Computer_Vision/Exports/output_video.mp4"
frames_to_delete = Frames_to_Delete
delete_frames(input_video_path, output_video_path, frames_to_delete)


Delete only Frames Which Are Staganent For Few mintues

In [ ]:
import cv2

def find_consecutive_sequence_indices(numbers):
    start_index = 0
    end_index = 0
    result = []

    for i in range(1, len(numbers)):
        if numbers[i] == numbers[i - 1] + 1:
            end_index = i
        else:
            if end_index - start_index > 0:
                result.append((start_index, end_index))
            start_index = i
            end_index = i

    # Check if the last sequence is consecutive
    if end_index - start_index > 0:
        result.append((start_index, end_index))

    return result

def delete_frames_in_sequence(input_video_path, output_video_path, sequence_indices):
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    frame_number = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Check if the current frame number is within any of the sequence ranges
        for start, end in sequence_indices:
            if start <= frame_number <= end:
                break  # Skip writing the frame if it falls within a sequence
        else:
            out.write(frame)

        frame_number += 1

    cap.release()
    out.release()

# Example usage
input_video_path = "/Users/jatavathpavannaik/Documents/python/Computer_Vision/Bookin.com_Staganent.mov"
output_video_path = "/Users/jatavathpavannaik/Documents/python/Computer_Vision/Exports/output_video.mp4"
numbers=desired_df["Frame"].tolist()
sequence_indices = find_consecutive_sequence_indices(numbers)
delete_frames_in_sequence(input_video_path, output_video_path, sequence_indices)
